In [8]:
import tinvest
import pandas as pd
from configparser import ConfigParser
from datetime import datetime, timedelta

In [9]:
config_parser = ConfigParser()

In [10]:
config_parser.read("../airflow/tinkoff.cfg")

['../airflow/tinkoff.cfg']

In [11]:
token = config_parser.get("core", "TOKEN_TINKOFF")

In [12]:
client = tinvest.SyncClient(token, use_sandbox=True)

In [13]:
client.clear_sandbox_account()

Empty(payload={}, status='Ok', tracking_id='72a5219d44292522')

In [14]:
client.get_portfolio_currencies()

PortfolioCurrenciesResponse(payload=Currencies(currencies=[CurrencyPosition(balance=Decimal('0'), blocked=None, currency=<Currency.eur: 'EUR'>), CurrencyPosition(balance=Decimal('0'), blocked=None, currency=<Currency.rub: 'RUB'>), CurrencyPosition(balance=Decimal('0'), blocked=None, currency=<Currency.usd: 'USD'>)]), status='Ok', tracking_id='cd5327ceddb5d707')

In [8]:
client.set_sandbox_currencies_balance(
    tinvest.schemas.SandboxSetCurrencyBalanceRequest(
        balance=100000,
        currency=tinvest.schemas.SandboxCurrency('USD')
    )
)

Empty(payload={}, status='Ok', tracking_id='851b54d243017d25')

In [9]:
aapl = client.get_market_search_by_ticker('AAPL')

In [10]:
aapl

MarketInstrumentListResponse(payload=MarketInstrumentList(instruments=[MarketInstrument(currency=<Currency.usd: 'USD'>, figi='BBG000B9XRY4', isin='US0378331005', lot=1, min_price_increment=Decimal('0.01'), name='Apple', ticker='AAPL', type=<InstrumentType.stock: 'Stock'>, min_quantity=None)], total=Decimal('1')), status='Ok', tracking_id='9b1de1795eaf5c60')

In [11]:
aapl_figi = aapl.payload.instruments[0].figi

In [12]:
data = client.get_market_candles(
    aapl_figi,
    datetime.now() - timedelta(days=365),
    datetime.now(),
    tinvest.schemas.CandleResolution.day
)

In [13]:
data.payload.candles

[Candle(c=Decimal('124.38'), figi='BBG000B9XRY4', h=Decimal('124.98'), interval=<CandleResolution.day: 'day'>, l=Decimal('123.05'), o=Decimal('123.69'), time=datetime.datetime(2020, 12, 8, 7, 0, tzinfo=datetime.timezone.utc), v=19517377),
 Candle(c=Decimal('121.78'), figi='BBG000B9XRY4', h=Decimal('125.95'), interval=<CandleResolution.day: 'day'>, l=Decimal('121.07'), o=Decimal('124.5'), time=datetime.datetime(2020, 12, 9, 7, 0, tzinfo=datetime.timezone.utc), v=30229870),
 Candle(c=Decimal('123.24'), figi='BBG000B9XRY4', h=Decimal('123.85'), interval=<CandleResolution.day: 'day'>, l=Decimal('120.01'), o=Decimal('121.45'), time=datetime.datetime(2020, 12, 10, 7, 0, tzinfo=datetime.timezone.utc), v=21004315),
 Candle(c=Decimal('122.41'), figi='BBG000B9XRY4', h=Decimal('123.7'), interval=<CandleResolution.day: 'day'>, l=Decimal('120.56'), o=Decimal('123.16'), time=datetime.datetime(2020, 12, 11, 7, 0, tzinfo=datetime.timezone.utc), v=21024768),
 Candle(c=Decimal('121.78'), figi='BBG000B9X

In [14]:
df = pd.DataFrame(
    data=(
        (
            candle.time,
            candle.o,
            candle.h,
            candle.l,
            candle.c,
            candle.v,

        ) for candle in data.payload.candles
    ),
    columns=(
        'time',
        'open',
        'high',
        'low',
        'close',
        'volume',
    )
)

In [15]:
df.tail()

,time,open,high,low,close,volume
247,2021-12-01 04:00:00+00:00,165.64,170.29,164.2,164.77,30766146
248,2021-12-02 04:00:00+00:00,166.56,166.56,157.8,163.76,30425416
249,2021-12-03 04:00:00+00:00,163.1,165.35,159.74,161.84,27492621
250,2021-12-06 04:00:00+00:00,161.7,167.87,161.7,165.32,26635166
251,2021-12-07 04:00:00+00:00,166.3,171.58,165.18,170.61,11292511


In [16]:
#### 2.Save to DB

In [17]:
import psycopg2
from io import StringIO
import csv

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   time    252 non-null    datetime64[ns, UTC]
 1   open    252 non-null    object             
 2   high    252 non-null    object             
 3   low     252 non-null    object             
 4   close   252 non-null    object             
 5   volume  252 non-null    int64              
dtypes: datetime64[ns, UTC](1), int64(1), object(4)
memory usage: 11.9+ KB


In [19]:
df.loc[:, 'open':'close'] = df.loc[:, 'open':'close'].astype('float16')

In [20]:
login = 'trader'
password = 'income'
host = 'localhost'
port = 5432
dbname = 'stocks'

In [21]:
DB_URL = f'user={login} password = {password} host = {host} port = {port} dbname = {dbname}'

In [22]:
def load_df_to_db(df: pd.DataFrame, table_name: str)->None:
    buffer = StringIO()
    df.to_csv(buffer, index=False, sep='|', na_rep='NUL', quoting=csv.QUOTE_MINIMAL,
             header=False, float_format='%.8f', doublequote=False, escapechar='\\')
    buffer.seek(0)
    copy_query = f"""
        COPY {table_name}({','.join(df.columns)})
        FROM STDIN
        DELIMITER '|'
        NULL 'NUL'
    """
    conn = psycopg2.connect(dsn=DB_URL)
    with conn.cursor() as cursor:
        cursor.copy_expert(copy_query, buffer)
    conn.commit()
    conn.close()

In [49]:
load_df_to_db(df, 'aapl')